# Introduction: Landmarks

In [ ]:
import deltascope as ds
import deltascope.alignment as ut

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize, Normalizer
from scipy.optimize import minimize
from scipy.stats import normaltest

import os
import tqdm
import json
import time

# Import data
The user needs to specify the directories containing the data of interest (after ilastik processing). Each sample type should have a key which corresponds to the directory path. Additionally, each object should have a list that includes the channels of interest.

In [ ]:
# --------------------------------
# -------- User input ------------
# --------------------------------

data = {
    # Specify first sample type name. ex: 'wt'
    'stype1': {
        # Specify path to data directory
        'path': 'path_to_data_prob_directory_for_first_sample_type',
        # Specify which channels are in the directory and are of interest
        'channels': ['ch1','ch2'] # ex: ['AT','ZRF']
    },
    # Specify second sample type name. ex: 'yot'
    'stype2': {
        # Specify path to data directory
        'path': 'path_to_data_prob_directory_for_second_sample_type',
        # Specify which channels are in the directory and are of interest
        'channels': ['ch1','ch2'] # ex: ['AT','ZRF']
    }
}

We'll generate a list of pairs of stypes and channels for ease of use.

In [ ]:
data_pairs = []
for s in data.keys():
    for c in data[s]['channels']:
        data_pairs.append((s,c))

We can now read in all datafiles specified by the `data` dictionary above.

In [ ]:
D = {}
for s in data.keys():
    D[s] = {}
    for c in data[s]['channels']:
        D[s][c] = ds.read_psi_to_dict(data[s]['path'],c)

# Calculate optimum landmark parameters
Choose a single sample type and channel which will be used as the control to calculate the appropriate size of landmark bins. Typically we use wildtype samples and the AT (structural) channel. We will run a parameter sweep of bin sizes using ds.anumSelect.

In [ ]:
# --------------------------------
# -------- User input ------------
# --------------------------------

# Specify sample type to use as control. ex: 'wt'
s_ctrl = 'control_sample_type'
# Specify channel to use as control. ex: 'AT'
c_ctrl = 'control_channel'

# Specify size of theta bins in radians
theta_step = np.pi/4

# Specify minimum a value when sweeping various alpha bin sizes
amn = 2
# Specify maximum a value
amx = 50
# Specify the step size in a value
astep = 3 

In [ ]:
optr = ds.anumSelect(D[s_ctrl][c_ctrl])
optr.param_sweep(theta_step, amn=amn, amx=amx, astep=astep,
                 rnull=np.nan, DT='r')

Plot result of the parameter sweep to get a sense of the results.

In [ ]:
# --------------------------------
# -------- User input ------------
# --------------------------------

# Select degrees of freedom for curve fitting
dof = 5

In [ ]:
x = np.arange(amn,amx,astep)
fig,ax = plt.subplots()

# Fit a curve to bin variance values
optrMbvarray = np.array(optr.Mbv).reshape(1, len(optr.Mbv))
pbv = np.polyfit(x, normalize(optrMbvarray)[0], dof)
fbv = np.poly1d(pbv)
ax.plot(x, fbv(x), c='b', label='Bin Variance')

# Fit curve to sample variance values
optrMsvarray = np.array(optr.Msv).reshape(1, len(optr.Msv))
psv = np.polyfit(x, normalize(optrMsvarray)[0], dof)
fsv = np.poly1d(psv)
ax.plot(x, fsv(x), c='g', label='Sample Variance')

# Plot sum of sample and bin variances
ax.plot(x, fsv(x)+fbv(x), c='c', label='Total Variance')

ax.legend()
tstamp = datetime.datetime.now().strftime('%Y-%m-%d')
location = "directory_to_save_figure_to"  #MODIFY PATH HERE
fig.savefig(location+tstamp+'_yotlandmarkoptimization-{}-{}.png'.format(c_ctrl,s_ctrl))

Now we can calculate the optimal bin size with the help of the user specifying a best guess for the optimal value.

In [ ]:
# --------------------------------
# -------- User input ------------
# --------------------------------

# Guess the approximate value on the horizontal axis where total variance is minimized
guess = 25

In [ ]:
opt = minimize(fbv+fsv, guess)
ax.axvline(opt.x, c='r', label='Optimum: '+str(np.round(opt.x[0], 2)))
print(opt.x[0])

# Calculate landmark bins
Based on the analysis above, we can select the optimal value of alpha bins.

In [ ]:
# --------------------------------
# -------- User input ------------
# --------------------------------

# Pick an integer value for bin number based on results above
anum = 25

# Specify the percentile(s) which will be used to calculate landmarks
percbins = [50]

Calculate landmark bins based on user input parameters and the previously specified control sample.

In [ ]:
lm = ds.landmarks(percbins=percbins, rnull=np.nan)
lm.calc_bins(D[s_ctrl][c_ctrl], anum, theta_step)

print('Alpha bins')
print(lm.acbins)
print('Theta bins')
print(lm.tbins)

# Calculate landmarks

In [ ]:
lmdf = pd.DataFrame()

# Loop through each pair of stype and channels
for s,c in tqdm.tqdm(data_pairs):
    print(s,c)
    # Calculate landmarks for each sample with this data pair
    for k,df in tqdm.tqdm(D[s][c].items()):
        lmdf = lm.calc_perc(df, k, '-'.join([s,c]), lmdf)
        
# Set timestamp for saving data
tstamp = time.strftime("%m-%d-%H-%M",time.localtime())
        
# Save completed landmarks to a csv file
lmdf.to_csv(tstamp+'_landmarks.csv')
print('Landmarks saved to csv')

# Save landmark bins to json file
bins = {
    'acbins':list(lm.acbins),
    'tbins':list(lm.tbins)
}
with open(tstamp+'_landmarks_bins.json', 'w') as outfile:
    json.dump(bins, outfile)
print('Bins saved to json')